#### Jakub Zaręba

In [3]:
# Korzystam z mozliwosci pobrania danych przy uzyciu biblioteki ucimlrepo dla ułatwienia
from ucimlrepo import fetch_ucirepo 
import pandas as pd

communities_and_crime = fetch_ucirepo(id=183) 

In [4]:
X = communities_and_crime.data.features
y=  communities_and_crime.data.targets
alldata=pd.concat([X,y],axis=1)

print(alldata[:5])

   state county community        communityname  fold  population  \
0      8      ?         ?         Lakewoodcity     1        0.19   
1     53      ?         ?          Tukwilacity     1        0.00   
2     24      ?         ?         Aberdeentown     1        0.00   
3     34      5     81440  Willingborotownship     1        0.04   
4     42     95      6096    Bethlehemtownship     1        0.01   

   householdsize  racepctblack  racePctWhite  racePctAsian  ...  LandArea  \
0           0.33          0.02          0.90          0.12  ...      0.12   
1           0.16          0.12          0.74          0.45  ...      0.02   
2           0.42          0.49          0.56          0.17  ...      0.01   
3           0.77          1.00          0.08          0.12  ...      0.02   
4           0.55          0.02          0.95          0.09  ...      0.04   

   PopDens  PctUsePubTrans  PolicCars  PolicOperBudg  LemasPctPolicOnPatr  \
0     0.26            0.20       0.06           0.0

Jak można zauważyć, w wykorzystywanym zbiorze danych wartości nieznane są oznaczane przy użyciu "?". Aby usunąć wartości nieznane, to dla ułatwienia najpierw zamienię każde wystąpienie "?" na NaN, a następnie skorzystam z domyślnych funkcji do pozbycia się NaN. Pozbędę się od razu kolumny "communityname", która jest typu String oraz nie będzie potrzebna.

In [5]:
for col in alldata.columns:
    alldata[col]=alldata[col].apply(lambda x: x if x!='?' else pd.NA)
alldata=alldata.drop(columns=["communityname"])

Sprawdźmy ile mamy wartości "NaN" w danych, dla każdej kolumny.

In [6]:
columns_with_nan = [(col, alldata[col].isna().sum()) for col in alldata.columns if alldata[col].isna().sum() > 0]
columns_with_nan.sort(key=lambda x: x[1], reverse=True)
for col, nan_count in columns_with_nan:
    print(f"{col}: {nan_count}")

LemasSwornFT: 1675
LemasSwFTPerPop: 1675
LemasSwFTFieldOps: 1675
LemasSwFTFieldPerPop: 1675
LemasTotalReq: 1675
LemasTotReqPerPop: 1675
PolicReqPerOffic: 1675
PolicPerPop: 1675
RacialMatchCommPol: 1675
PctPolicWhite: 1675
PctPolicBlack: 1675
PctPolicHisp: 1675
PctPolicAsian: 1675
PctPolicMinor: 1675
OfficAssgnDrugUnits: 1675
NumKindsDrugsSeiz: 1675
PolicAveOTWorked: 1675
PolicCars: 1675
PolicOperBudg: 1675
LemasPctPolicOnPatr: 1675
LemasGangUnitDeploy: 1675
PolicBudgPerPop: 1675
community: 1177
county: 1174
OtherPerCap: 1


Usuwam kolumny, które zawierają wartości NaN.

In [7]:
alldata=alldata.dropna(axis=1)

Sprawdzam, czy wszystkie kolumny mają tylko i wyłącznie wartości numeryczne:

In [9]:
print(all(alldata[col].dtype in ['float64', 'int64'] for col in alldata.columns))

True


Teraz podzielę zbiór na zbiory uczące oraz testowe, dokonam skalowania zbioru uczącego przy użyciu StandardScaler oraz stworzę modele.

In [10]:
Y=alldata["ViolentCrimesPerPop"]
X=alldata.drop(columns=["ViolentCrimesPerPop"])

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import root_mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=44)

# Skalowanie danych
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
# Na podstawie scalera wyuczonego na danych treningowych przekształcamy dane testowe
X_test=scaler.transform(X_test)

# Zwykły model regresji liniowej
linear_model=LinearRegression()
linear_model.fit(X_train,y_train)

# Model z regularyzacją, alpha została po prostu wymyślona
ridge_model=Ridge(alpha=4.7)
ridge_model.fit(X_train,y_train)

mse_linear=root_mean_squared_error(y_test,linear_model.predict(X_test))
mse_ridge=root_mean_squared_error(y_test,ridge_model.predict(X_test))

print(f"Metryka RMSE dla standardowego modelu regresji liniowej: {mse_linear}")
print(f"Metryka RMSE dla modelu z regularyzacją: {mse_ridge}")


Metryka RMSE dla standardowego modelu regresji liniowej: 0.1307543569484191
Metryka RMSE dla modelu z regularyzacją: 0.12945938724912343
